In [6]:
import numpy as np
import cv2  
import torch
import os

In [41]:

def resize_mask(mask, shape):
    return cv2.resize(mask, (shape[1], shape[0]), interpolation=cv2.INTER_LINEAR)

def convert_to_grayscale(mask):
    if len(mask.shape) == 3 and mask.shape[2] == 3:
        grayscale_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        return grayscale_mask
    return mask

def calculate_iou(mask1, mask2):
    target_shape = max(mask1.shape, mask2.shape, key=lambda x: x[0] * x[1])

    
    resized_mask1 = resize_mask(mask1, target_shape)
    resized_mask2 = resize_mask(mask2, target_shape)
    
    
    intersection = np.logical_and(resized_mask1, resized_mask2).sum()
    union = np.logical_or(resized_mask1, resized_mask2).sum()
    iou = intersection / union if union != 0 else 0
    
    return iou

In [7]:
def load_tensors_from_folder(folder_path):
    tensor_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.pth')])
    tensors = {}
    for file in tensor_files:
        tensor_path = os.path.join(folder_path, file)
        tensors[file] = torch.load(tensor_path)
    return tensors

In [9]:
tensors1 = load_tensors_from_folder('./lime_masks')
tensors2 = load_tensors_from_folder('./gradcam_masks')
    
# Ensure both folders contain the same files
assert tensors1.keys() == tensors2.keys(), "Folders do not contain the same files"

In [42]:
for file in tensors1:
    mask1 = tensors1[file].astype('float32')
    mask2 = tensors2[file].astype('float32')
    mask2 = convert_to_grayscale(mask2)
    iou = calculate_iou(mask1, mask2)
    print(f"IoU for {file}: {iou}")

IoU for n01443537_goldfish.pth: 0.18485815007554138
IoU for n01491361_tiger_shark.pth: 0.2273188830306581
IoU for n01608432_kite.pth: 0.17093506230371797
IoU for n01616318_vulture.pth: 0.18044872677219545
IoU for n01677366_common_iguana.pth: 0.1405877591401718
IoU for n02007558_flamingo.pth: 0.1283885668330932
IoU for n02018207_American_coot.pth: 0.24490890853530523
IoU for n02098286_West_Highland_white_terrier.pth: 0.19151308674627235
IoU for n04037443_racer.pth: 0.13805013725026813
IoU for n07747607_orange.pth: 0.21358051636260927
